In [229]:
import psycopg2
import pandas as pd


In [230]:
def create_database():
    conn=psycopg2.connect("host=localhost dbname=postgres user=postgres password=Sameen@2000")
    conn.set_session(autocommit=True)
    cur=conn.cursor()

    cur.execute("""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = 'account'
        AND pid <> pg_backend_pid();
    """)

    cur.execute("drop database if exists account")
    cur.execute("create database account")

    conn.close()
    conn=psycopg2.connect("host=localhost dbname=account user=postgres password=Sameen@2000")
    cur=conn.cursor()
    return cur, conn





In [231]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [232]:
def create_tables(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [233]:
accountcountry=pd.read_excel('country.xlsx')

In [234]:
accountcountry.head()

,country_name,continent,disease,population,literacy_r,life_exp,area
0,Nepal,Asia,covid,2500000,0.45,55,1412566
1,India,Asia,covid,5000000,0.55,55,"45,65,0000"
2,UK,Europe,covid,3000000,0.50,70,"6,00,000"
3,Canada,North America,covid,4000000,0.60,70,"56,78.890"
4,Australia,Oceania,covid,350000,0.60,80,"10,00,3452"


In [235]:
accountcountry_clean=accountcountry[['country_name', 'disease', 'population', 'literacy_r', 'life_exp']]

In [236]:
accountcountry_clean.head()

,country_name,disease,population,literacy_r,life_exp
0,Nepal,covid,2500000,0.45,55
1,India,covid,5000000,0.55,55
2,UK,covid,3000000,0.50,70
3,Canada,covid,4000000,0.60,70
4,Australia,covid,350000,0.60,80


In [237]:
accountcovid=pd.read_excel('covid.xlsx')

In [238]:
accountcovid.columns

Index(['country_name', 'p_ID', 'hospital', 'duration', 'age', 'death',
       'vaccine', 'amount$'],
      dtype='object')

In [239]:
accountcovid=accountcovid.drop(['amount$', 'death'], axis=1)

In [240]:
accountcovid.head()

,country_name,p_ID,hospital,duration,age,vaccine
0,Nepal,111,Grand,10,20,moderna
1,India,112,merry,11,24,jansen
2,UK,113,santa,12,55,jansen
3,Canada,114,curryili,11,40,moderna
4,Australia,115,mercury,12,32,moderna


In [241]:
accountpatient=pd.read_excel('patient.xlsx')

In [242]:
accountpatient.head()

,patient_name,p_id,occupation,married,adress,country,height,weight
0,abascus,111,student,no,ab,nepal,6',45
1,hibris,112,coder,no,cd,india,7',45
2,corny,113,student,yes,ef,Uk,67',56
3,lily,114,doctor,yes,gh,Canada,56',67
4,haxe,115,enginer,yes,ij,Australia,45',78


In [243]:
accountpatient=accountpatient.drop(['adress', 'height'], axis=1)

In [244]:
accountpatient.head()

,patient_name,p_id,occupation,married,country,weight
0,abascus,111,student,no,nepal,45
1,hibris,112,coder,no,india,45
2,corny,113,student,yes,Uk,56
3,lily,114,doctor,yes,Canada,67
4,haxe,115,enginer,yes,Australia,78


In [245]:
cur, conn = create_database()

In [246]:
country_create_table=("""CREATE TABLE IF NOT EXISTS accountcountry(
country_name VARCHAR PRIMARY KEY,
disease VARCHAR,
population BIGINT,
literacy_r float,
life_exp int
)""")


In [247]:
cur.execute(country_create_table)
conn.commit()

In [248]:
accountcovid_create_table=("""CREATE TABLE IF NOT EXISTS accountcovid(
country_name VARCHAR ,
P_id int,
hospital VARCHAR,
duration int,
age int,
vaccine VARCHAR
)""")


In [249]:
cur.execute(accountcovid_create_table)
conn.commit()

In [250]:
accountpatient_create_table=("""CREATE TABLE IF NOT EXISTS accountpatient(
patient_name VARCHAR,
p_id int,
occupation VARCHAR,
married VARCHAR,
country VARCHAR,
weight int
)""")


In [251]:
cur.execute(accountpatient_create_table)
conn.commit()

In [252]:
country_create_table_insert = ("""
INSERT INTO accountcountry(
    country_name,
    disease,
    population,
    literacy_r,
    life_exp)
VALUES(%s, %s, %s, %s, %s)
""")


In [253]:
for i, row in accountcountry_clean.iterrows():
    cur.execute(country_create_table_insert, tuple(row))


In [254]:
covid_create_table_insert = ("""
INSERT INTO accountcovid(
    country_name,
    p_id,
    hospital,
    duration,
    age,
    vaccine)
VALUES(%s, %s, %s, %s, %s, %s)
""")

In [256]:
for i, row in accountcovid.iterrows():
    cur.execute(covid_create_table_insert, tuple(row))

In [257]:
patient_create_table_insert = ("""
INSERT INTO accountpatient(
    patient_name,
    p_id,
    occupation,
    married,
    country,
    weight)
VALUES(%s, %s, %s, %s, %s, %s)
""")

In [258]:
for i, row in accountpatient.iterrows():
    cur.execute(patient_create_table_insert, tuple(row))

In [259]:
conn.commit()